In [ ]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.7 MB/s eta 0:00:00


In [ ]:
pip install gtts


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [ ]:
# app.py
# A complete, integrated Career AI application with a Streamlit frontend.

# --- 1. Installation ---
# pip install streamlit transformers torch datasets requests gtts

import streamlit as st
import os
import json
import requests
from gtts import gTTS
from datetime import datetime
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    TextDataset,
    DataCollatorForLanguageModeling,
)

# --- App Configuration ---
st.set_page_config(page_title="Career AI Assistant", layout="wide")

# --- Global Variables & API Keys ---
OPENROUTER_API_KEY = "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"
MODEL_OUTPUT_DIR = "./career-ai-roadmap-model"

# --- Caching for Model Training ---
# This ensures the model is only fine-tuned once per session.
@st.cache_resource
def fine_tune_roadmap_model():
    """Creates a dataset and fine-tunes the distilgpt2 model for roadmap generation."""
    st.info("Fine-tuning the base model for roadmap generation... (This happens once per session)")

    roadmap_dataset_content = [
        {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Beginner. ROADMAP: ### Beginner Roadmap for Backend Developer\n\n**Month 1: Core Concepts**\n- **Courses:** Complete 'Introduction to Web Development'. Study HTTP protocols.\n- **Project:** Build a command-line weather app using a public API."},
        {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Intermediate. ROADMAP: ### Intermediate Roadmap for Backend Developer\n\n**Month 1: Authentication & Security**\n- **Courses:** Study JWT and OAuth2.\n- **Project:** Implement user registration and login in an application."},
        {"text": "PROFILE: Priya Singh, Computer Science, 2024, Python, Java, SQL. LEVEL: Advanced. ROADMAP: ### Advanced Roadmap for Backend Developer\n\n**Month 1: System Design & Scalability**\n- **Courses:** Study load balancing and message queues.\n- **Project:** Design a scalable architecture for a social media app."},
    ]
    dataset_filename = "roadmap_data.jsonl"
    with open(dataset_filename, 'w') as f:
        for entry in roadmap_dataset_content:
            f.write(json.dumps(entry) + '\n')

    tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model = AutoModelForCausalLM.from_pretrained("distilgpt2")
    model.resize_token_embeddings(len(tokenizer))

    train_dataset = TextDataset(tokenizer=tokenizer, file_path=dataset_filename, block_size=256)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
    training_args = TrainingArguments(
        output_dir=MODEL_OUTPUT_DIR, overwrite_output_dir=True, num_train_epochs=15,
        per_device_train_batch_size=1, save_steps=50, save_total_limit=2, logging_steps=10,
    )
    trainer = Trainer(model=model, args=training_args, data_collator=data_collator, train_dataset=train_dataset)

    with st.spinner("Training... Please wait."):
        trainer.train()
        trainer.save_model(MODEL_OUTPUT_DIR)
        tokenizer.save_pretrained(MODEL_OUTPUT_DIR)

    st.success("Model fine-tuning complete!")
    return True

# --- All Generation Modules ---

def generate_roadmap(profile_text, experience_level):
    loaded_tokenizer = AutoTokenizer.from_pretrained(MODEL_OUTPUT_DIR)
    loaded_model = AutoModelForCausalLM.from_pretrained(MODEL_OUTPUT_DIR)
    prompt = f"PROFILE: {profile_text}. LEVEL: {experience_level}. ROADMAP:"
    inputs = loaded_tokenizer(prompt, return_tensors="pt")
    outputs = loaded_model.generate(
        inputs.input_ids, max_length=300, num_return_sequences=1,
        pad_token_id=loaded_tokenizer.pad_token_id, eos_token_id=loaded_tokenizer.eos_token_id,
        do_sample=True, temperature=0.7, top_k=50
    )
    generated_text = loaded_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text.split("ROADMAP:")[1].strip()

def generate_course_content(profile_text, topic, api_key):
    prompt = f'Generate a complete lesson on "{topic}" for a student with profile: "{profile_text}". Include Introduction, Core Concepts, Code Examples, and Conclusion.'
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"},
            data=json.dumps({"model": "deepseek/deepseek-r1-0528-qwen3-8b:free", "messages": [{"role": "user", "content": prompt}]})
        )
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e: return f"Error: {e}"

def create_voice_content(text, filename="course_audio.mp3"):
    try:
        tts = gTTS(text=text, lang='en')
        tts.save(filename)
        return filename
    except Exception as e: return None

def generate_certificate(profile_name, course_topic):
    date = datetime.now().strftime("%B %d, %Y")
    return f"CERTIFICATE OF COMPLETION\n\nPresented to: {profile_name}\nFor completing: {course_topic}\nIssued on: {date}"

def suggest_linkedin_post(profile_name, course_topic, api_key):
    prompt = f"Write an engaging LinkedIn post for '{profile_name}' who completed a course on '{course_topic}'. Include skills and hashtags."
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"},
            data=json.dumps({"model": "deepseek/deepseek-r1-0528-qwen3-8b:free", "messages": [{"role": "user", "content": prompt}]})
        )
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e: return f"Error: {e}"

def suggest_project(profile_text, roadmap_text, api_key):
    prompt = f"Suggest a capstone project for this profile and roadmap.\nPROFILE: {profile_text}\nROADMAP: {roadmap_text}"
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"},
            data=json.dumps({"model": "deepseek/deepseek-r1-0528-qwen3-8b:free", "messages": [{"role": "user", "content": prompt}]})
        )
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e: return f"Error: {e}"

def generate_project_plan(project_suggestion, api_key):
    prompt = f"Create a step-by-step plan to build this project: {project_suggestion}. Include Tech Stack, MVP Features, Build Guide, and Next Steps."
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"},
            data=json.dumps({"model": "deepseek/deepseek-r1-0528-qwen3-8b:free", "messages": [{"role": "user", "content": prompt}]})
        )
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
    except Exception as e: return f"Error: {e}"

def generate_latex_resume(profile_text, api_key):
    name, major, year, skills = [p.strip() for p in profile_text.split(',')]
    prompt = f"Create a complete, single-file LaTeX resume (article class). Profile: {name}, {major}, {year}, Skills: {skills}. Include Header, Education, Skills, 2 Projects, 1 Experience. Output only raw LaTeX code."
    try:
        response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"},
            data=json.dumps({"model": "deepseek/deepseek-r1-0528-qwen3-8b:free", "messages": [{"role": "user", "content": prompt}]})
        )
        response.raise_for_status()
        latex_code = response.json()['choices'][0]['message']['content']
        if "```latex" in latex_code:
            latex_code = latex_code.split("```latex\n")[1].split("```")[0]
        return latex_code
    except Exception as e: return f"Error: {e}"

# --- Frontend Application ---
st.title("🎓 AI Career Assistant")
st.markdown("Your all-in-one platform for personalized career development, powered by Generative AI.")

# Initialize fine-tuning
fine_tune_roadmap_model()

# --- User Profile Input ---
st.sidebar.header("Your Profile")
profile_text = st.sidebar.text_area(
    "Enter your profile (Name, Major, Year, Skills)",
    "Priya Singh, Computer Science, 2024, Python, Java, SQL"
)
experience_level = st.sidebar.selectbox(
    "Select Your Experience Level",
    ("Beginner", "Intermediate", "Advanced")
)

# --- Main Application Tabs ---
tab1, tab2, tab3, tab4, tab5 = st.tabs(["🗺️ Roadmap", "📚 Course Content", "🏆 Certificate & Social", "🛠️ Project Plan", "📄 LaTeX Resume"])

if 'roadmap' not in st.session_state:
    st.session_state.roadmap = ""

with tab1:
    st.header("Personalized Career Roadmap")
    if st.button("Generate Roadmap"):
        with st.spinner("Generating your personalized roadmap..."):
            st.session_state.roadmap = generate_roadmap(profile_text, experience_level)
    if st.session_state.roadmap:
        st.markdown(st.session_state.roadmap)

with tab2:
    st.header("Generative Course Content")
    course_topic = st.text_input("Enter a topic from your roadmap", "Docker & Containers")
    if st.button("Generate Course"):
        with st.spinner(f"Generating course content for '{course_topic}'..."):
            course_text = generate_course_content(profile_text, course_topic, OPENROUTER_API_KEY)
            st.session_state.course_text = course_text

            audio_file = create_voice_content(course_text)
            st.session_state.audio_file = audio_file

    if 'course_text' in st.session_state:
        st.markdown(st.session_state.course_text)
        if st.session_state.get('audio_file'):
            st.audio(st.session_state.audio_file)

with tab3:
    st.header("Certificate and LinkedIn Post")
    if st.button("Generate Certificate & Post"):
        profile_name = profile_text.split(',')[0]
        with st.spinner("Generating your assets..."):
            st.session_state.certificate = generate_certificate(profile_name, course_topic)
            st.session_state.linkedin_post = suggest_linkedin_post(profile_name, course_topic, OPENROUTER_API_KEY)

    if 'certificate' in st.session_state:
        st.subheader("Certificate of Completion")
        st.code(st.session_state.certificate, language='text')
    if 'linkedin_post' in st.session_state:
        st.subheader("Suggested LinkedIn Post")
        st.markdown(st.session_state.linkedin_post)

with tab4:
    st.header("Capstone Project Plan")
    if st.button("Generate Project Plan"):
        if not st.session_state.roadmap:
            st.warning("Please generate a roadmap first.")
        else:
            with st.spinner("Generating project suggestion and plan..."):
                project_suggestion = suggest_project(profile_text, st.session_state.roadmap, OPENROUTER_API_KEY)
                st.session_state.project_suggestion = project_suggestion

                project_plan = generate_project_plan(project_suggestion, OPENROUTER_API_KEY)
                st.session_state.project_plan = project_plan

    if 'project_suggestion' in st.session_state:
        st.subheader("Project Suggestion")
        st.markdown(st.session_state.project_suggestion)
    if 'project_plan' in st.session_state:
        st.subheader("Project Build Plan")
        st.markdown(st.session_state.project_plan)

with tab5:
    st.header("ATS-Friendly LaTeX Resume")
    if st.button("Generate LaTeX Resume"):
        with st.spinner("Generating your resume..."):
            st.session_state.latex_resume = generate_latex_resume(profile_text, OPENROUTER_API_KEY)

    if 'latex_resume' in st.session_state:
        st.info("Copy the code below and paste it into an Overleaf project to compile your PDF resume.")
        st.code(st.session_state.latex_resume, language='latex')

2025-08-23 21:23:22.805 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 21:23:22.829 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 21:23:24.348 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-23 21:23:24.351 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 21:23:24.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 21:23:24.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 21:23:24.367 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/usr/local/lib/python3.12/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(
2025-08-23 21:23:56.617 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 21:23:56.621 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-23 21:23:56.625 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored w

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter: